Get my competitors' pricing from their websites

In [ ]:
competitor_sites = [
    {
        "name": "Connected Stories",
        "url": "https://www.connected-stories.com/pricing"
    },
    {
        "name": "ZBrain",
        "url": "https://zbrain.ai/pricing/"
    },
    {
        "name": "Pecan AI",
        "url": "https://www.pecan.ai/pricing/"
    },
    {
        "name": "Bolt Chat AI",
        "url": "https://www.boltchatai.com/pricing/"
    },
]